In [0]:

# Load the section metadatas
import json
import pandas as pd

with open("/Volumes/main/casml/raw_data/book-metadata.json", "r") as f:
    book_metadata = json.load(f)

# Map page-nums to sections
def map_page_to_section(metadata):
    page_map = {}
    for section, sec_data in metadata.items():
        sec_start = sec_data.get("page_start")
        sec_end = sec_data.get("page_end")
        subsections = sec_data.get("subsections", {})
        for page in range(sec_start, sec_end + 1):
            page_map[page] = {"section": section, "subsection": None}
        for subsec, subsec_data in subsections.items():
            sub_start = subsec_data.get("page_start")
            sub_end = subsec_data.get("page_end")
            for page in range(sub_start, sub_end + 1):
                page_map[page] = {"section": section, "subsection": subsec}
    return page_map

page_to_section = map_page_to_section(book_metadata)

# Convert page_to_section dict to a DataFrame
page_section_df = pd.DataFrame([
    {"page_num": page, "section": info["section"], "subsection": info["subsection"]}
    for page, info in page_to_section.items()
])

# Create Spark DataFrame
page_section_sdf = spark.createDataFrame(page_section_df)

# Create a temporary SQL table
page_section_sdf.createOrReplaceTempView("pag_to_subsection_temp")



In [0]:
%sql
USE main.casml;
CREATE TABLE IF NOT EXISTS knowledge_base (
  id BIGINT GENERATED ALWAYS AS IDENTITY,
  page_num INT,
  content STRING,
  subsection STRING
)
TBLPROPERTIES (delta.enableChangeDataFeed = true);

In [0]:
%sql
INSERT OVERWRITE TABLE knowledge_base (page_num, content)
SELECT
first_value(e.value:bbox[0]:page_id) AS page_num,
string_agg(e.value:content) AS content
FROM (
  SELECT ai_parse_document(content) AS parsed_document, path
  FROM READ_FILES('/Volumes/main/casml/raw_data/CoT.pdf', format => 'binaryFile')
) AS f,
LATERAL variant_explode(f.parsed_document:document:elements) AS e
GROUP BY e.value:bbox[0]:page_id::int, f.path


num_affected_rows,num_inserted_rows
43,43


In [0]:
%sql
MERGE INTO knowledge_base AS kb
USING pag_to_subsection_temp AS pts
ON kb.page_num = pts.page_num
WHEN MATCHED THEN
  UPDATE SET kb.subsection = pts.subsection;

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
36,36,0,0


In [0]:
%sql
SELECT * from knowledge_base

id page_num content subsection 9 0 Chain-of-Thought Prompting Elicits Reasoning in Large Language ModelsJason Wei Xuezhi Wang Dale Schuurmans Maarten BosmaBrian Ichter Fei Xia Ed H. Chi Quoc V. Le Denny ZhouY21 1905-6 [CS.5]Google Research, Brain Team
{jasonwei,dennyzhou}@google.comAbstractWe explore how generating a chain of thought—a series of intermediate reasoning steps—significantly improves the ability of large language models to perform complex reasoning. In particular, we show how such reasoning abilities emerge naturally in sufficiently large language models via a simple method called chain-of-thought prompting, where a few chain of thought demonstrations are provided as exemplars in prompting.Experiments on three large language models show that chain-of-thought prompting improves performance on a range of arithmetic, commonsense, and symbolic reasoning tasks. The empirical gains can be striking. For instance, prompting a PaLM 540B with just eight chain-of-thought exemplars achieves state-of-the-art accuracy on the GSM8K benchmark of math word problems, surpassing even finetuned GPT-3 with a verifier.Figure 1: Chain-of-thought prompting enables large language models to tackle complex arithmetic, commonsense, and symbolic reasoning tasks. Chain-of-thought reasoning processes are highlighted.36th Conference on Neural Information Processing Systems (NeurIPS 2022). null 11 4 Second, chain-of-thought prompting has larger performance gains for more-complicated problems. For instance, for GSM8K (the dataset with the lowest baseline performance), performance more than doubled for the largest GPT and PaLM models. On the other hand, for Sin-gleOp, the easiest subset of MAWPS which only requires a single step to solve, performance improvements were either negative or very small (see Appendix Table 3).Third, chain-of-thought prompting via GPT-3 175B and PaLM 540B compares favorably to prior state of the art, which typically finetunes a task-specific model on a labeled training dataset. Figure 4 shows how PaLM 540B uses chain-of-thought prompting to achieve new state of the art on GSM8K, SVAMP, and MAWPS (though note that standard prompting already passed the prior best for SVAMP). On the other two datasets, AQUA and ASDiv, PaLM with chain-of-thought prompting reaches within 2% of the state of the art (Appendix Table 2).To better understand why chain-of-thought prompting works, we manually examined model-generated chains of thought by LaMDA 137B for GSM8K. Of 50 random examples where the model returned the correct final answer, all of the generated chains of thought were also logically and mathematically correct except two that coincidentally arrived at the correct answer (see Appendix D.1, and Table 8 for examples of correct model-generated chains of thought). We also randomly examined 50 random samples for which the model gave the wrong answer. The summary of this analysis is that 46% of the chains of thought were almost correct, barring minor mistakes (calculator error, symbol mapping error, or one reasoning step missing), and the errors in semantic understanding or coherence why scaling improves chain-of-thought reasoning made by PaLM 62B and whether those errors is that scaling PaLM to 540B fixes a large portion3.3 Ablation StudyThe observed benefits of using chain-of-thought prompting raises the natural question of whether the same performance improvements can be conferred via other types of prompting. Figure 5 shows an ablation study with three variations of chain of thought described below.Equation only. One reason for why chain-of-thought prompting might help is that it produces the mathematical equation to be evaluated, and so we test a variation where the model is prompted to output only a mathematical equation before giving the answer. Figure 5 shows that equation only prompting does not help much for GSM8K, which implies that the semantics of the questions in GSM8K are too challenging to directly translate into an equ

In [0]:
%pip install -U -qqqq mlflow>=3.1.1 langchain langgraph databricks-langchain pydantic databricks-agents unitycatalog-langchain[databricks] uv databricks-feature-engineering==0.12.1
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import time

def endpoint_exists(vsc, vs_endpoint_name):
  try:
    return vs_endpoint_name in [e['name'] for e in vsc.list_endpoints().get('endpoints', [])]
  except Exception as e:
    #Temp fix for potential REQUEST_LIMIT_EXCEEDED issue
    if "REQUEST_LIMIT_EXCEEDED" in str(e):
      print("WARN: couldn't get endpoint status due to REQUEST_LIMIT_EXCEEDED error. The demo will consider it exists")
      return True
    else:
      raise e

def wait_for_vs_endpoint_to_be_ready(vsc, vs_endpoint_name):
  for i in range(180):
    try:
      endpoint = vsc.get_endpoint(vs_endpoint_name)
    except Exception as e:
      #Temp fix for potential REQUEST_LIMIT_EXCEEDED issue
      if "REQUEST_LIMIT_EXCEEDED" in str(e):
        print("WARN: couldn't get endpoint status due to REQUEST_LIMIT_EXCEEDED error. Please manually check your endpoint status")
        return
      else:
        raise e
    status = endpoint.get("endpoint_status", endpoint.get("status"))["state"].upper()
    if "ONLINE" in status:
      return endpoint
    elif "PROVISIONING" in status or i <6:
      if i % 20 == 0: 
        print(f"Waiting for endpoint to be ready, this can take a few min... {endpoint}")
      time.sleep(10)
    else:
      raise Exception(f'''Error with the endpoint {vs_endpoint_name}. - this shouldn't happen: {endpoint}.\n Please delete it and re-run the previous cell: vsc.delete_endpoint("{vs_endpoint_name}")''')
  raise Exception(f"Timeout, your endpoint isn't ready yet: {vsc.get_endpoint(vs_endpoint_name)}")

def index_exists(vsc, endpoint_name, index_full_name):
    try:
        vsc.get_index(endpoint_name, index_full_name).describe()
        return True
    except Exception as e:
        if 'RESOURCE_DOES_NOT_EXIST' not in str(e):
            print(f'Unexpected error describing the index. This could be a permission issue.')
            raise e
    return False
  
def wait_for_index_to_be_ready(vsc, vs_endpoint_name, index_name):
  for i in range(180):
    idx = vsc.get_index(vs_endpoint_name, index_name).describe()
    index_status = idx.get('status', idx.get('index_status', {}))
    status = index_status.get('detailed_state', index_status.get('status', 'UNKNOWN')).upper()
    url = index_status.get('index_url', index_status.get('url', 'UNKNOWN'))
    if "ONLINE" in status:
      return
    if "UNKNOWN" in status:
      print(f"Can't get the status - will assume index is ready {idx} - url: {url}")
      return
    elif "PROVISIONING" in status:
      if i % 40 == 0: print(f"Waiting for index to be ready, this can take a few min... {index_status} - pipeline url:{url}")
      time.sleep(10)
    else:
        raise Exception(f'''Error with the index - this shouldn't happen. DLT pipeline might have been killed.\n Please delete it and re-run the previous cell: vsc.delete_index("{index_name}, {vs_endpoint_name}") \nIndex details: {idx}''')
  raise Exception(f"Timeout, your index isn't ready yet: {vsc.get_index(index_name, vs_endpoint_name)}")

In [0]:
VECTOR_SEARCH_ENDPOINT_NAME = "casml_vs_endpoint"

from databricks.vector_search.client import VectorSearchClient
vsc = VectorSearchClient(disable_notice=True)

if not endpoint_exists(vsc, VECTOR_SEARCH_ENDPOINT_NAME):
    endpoints = vsc.list_endpoints()
    if len(endpoints):
        endpoint_names = [ep['name'] for ep in endpoints['endpoints']]
        for name in endpoint_names:
            vsc.delete_endpoint(name)

    vsc.create_endpoint(name=VECTOR_SEARCH_ENDPOINT_NAME, endpoint_type="STANDARD")

wait_for_vs_endpoint_to_be_ready(vsc, VECTOR_SEARCH_ENDPOINT_NAME)
print(f"Endpoint named {VECTOR_SEARCH_ENDPOINT_NAME} is ready.")

Waiting for endpoint to be ready, this can take a few min... {'name': 'casml_vs_endpoint', 'creator': 'jmendozais@gmail.com', 'creation_timestamp': 1759515491775, 'last_updated_timestamp': 1759515491775, 'endpoint_type': 'STANDARD', 'last_updated_user': 'jmendozais@gmail.com', 'id': '47dc94ca-3f29-465f-9abe-df8a852e75ae', 'endpoint_status': {'state': 'PROVISIONING'}, 'num_indexes': 0}
Waiting for endpoint to be ready, this can take a few min... {'name': 'casml_vs_endpoint', 'creator': 'jmendozais@gmail.com', 'creation_timestamp': 1759515491775, 'last_updated_timestamp': 1759515491775, 'endpoint_type': 'STANDARD', 'last_updated_user': 'jmendozais@gmail.com', 'id': '47dc94ca-3f29-465f-9abe-df8a852e75ae', 'endpoint_status': {'state': 'PROVISIONING'}, 'num_indexes': 0}
Waiting for endpoint to be ready, this can take a few min... {'name': 'casml_vs_endpoint', 'creator': 'jmendozais@gmail.com', 'creation_timestamp': 1759515491775, 'last_updated_timestamp': 1759515491775, 'endpoint_type': 'ST

In [0]:
from databricks.sdk import WorkspaceClient

#The table we'd like to index
catalog = "main"
dbName = "casml"
source_table_fullname = f"{catalog}.{dbName}.knowledge_base"
# Where we want to store our index
vs_index_fullname = f"{catalog}.{dbName}.knowledge_base_vs_index"

if not index_exists(vsc, VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname):
  print(f"Creating index {vs_index_fullname} on endpoint {VECTOR_SEARCH_ENDPOINT_NAME}...")
  vsc.create_delta_sync_index(
    endpoint_name=VECTOR_SEARCH_ENDPOINT_NAME,
    index_name=vs_index_fullname,
    source_table_name=source_table_fullname,
    pipeline_type="TRIGGERED",
    primary_key="id",
    embedding_source_column='content', #The column containing our text
    embedding_model_endpoint_name='databricks-gte-large-en' #The embedding endpoint used to create the embeddings
  )
  #Let's wait for the index to be ready and all our embeddings to be created and indexed
  wait_for_index_to_be_ready(vsc, VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname)
else:
  #Trigger a sync to update our vs content with the new data saved in the tableoug
  wait_for_index_to_be_ready(vsc, VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname)
  vsc.get_index(VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname).sync()

print(f"index {vs_index_fullname} on table {source_table_fullname} is ready")

Creating index main.casml.knowledge_base_vs_index on endpoint casml_vs_endpoint...
Waiting for index to be ready, this can take a few min... {'detailed_state': 'PROVISIONING_INDEX', 'message': 'Delta sync Index creation is pending. Check latest status: https://dbc-4f84a5d3-c8b8.cloud.databricks.com/explore/data/main/casml/knowledge_base_vs_index', 'indexed_row_count': 0, 'ready': False, 'index_url': 'dbc-4f84a5d3-c8b8.cloud.databricks.com/api/2.0/vector-search/indexes/main.casml.knowledge_base_vs_index'} - pipeline url:dbc-4f84a5d3-c8b8.cloud.databricks.com/api/2.0/vector-search/indexes/main.casml.knowledge_base_vs_index
index main.casml.knowledge_base_vs_index on table main.casml.knowledge_base is ready


In [0]:
question = "What was the main contribution the 'Chain-of-Thought Prompting Elicits Reasoning in Large Language Models' paper"

results = vsc.get_index(VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname).similarity_search(
  query_text=question,
  columns=["id", "content"],
  num_results=1)
  
docs = results.get('result', {}).get('data_array', [])

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.


[[9.0,
  'Chain-of-Thought Prompting Elicits Reasoning in Large Language ModelsJason Wei Xuezhi Wang Dale Schuurmans Maarten BosmaBrian Ichter Fei Xia Ed H. Chi Quoc V. Le Denny ZhouY21 1905-6 [CS.5]Google Research, Brain Team\n{jasonwei,dennyzhou}@google.comAbstractWe explore how generating a chain of thought—a series of intermediate reasoning steps—significantly improves the ability of large language models to perform complex reasoning. In particular, we show how such reasoning abilities emerge naturally in sufficiently large language models via a simple method called chain-of-thought prompting, where a few chain of thought demonstrations are provided as exemplars in prompting.Experiments on three large language models show that chain-of-thought prompting improves performance on a range of arithmetic, commonsense, and symbolic reasoning tasks. The empirical gains can be striking. For instance, prompting a PaLM 540B with just eight chain-of-thought exemplars achieves state-of-the-art 